In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

from src.ksd.ksd import KSD
from src.ksd.kernel import IMQ, RBF, Linear

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm, trange
import pickle

import high_dim_power as hd

import matplotlib as mpl
import matplotlib.pyplot as pyplot
mpl.rcParams["mathtext.fontset"] = "cm"

In [3]:
tf.random.set_seed(1)

DELTA = 2. # 0.1

BANDWIDTH_POWER = 1.

STATISTIC = "mmd" # "ksd" # 

KERNEL = RBF 

DIR = "../res/high_dim"
SUFFIX = f"delta{DELTA}_r{BANDWIDTH_POWER}_{STATISTIC}_{KERNEL.__name__}"

EXTRA = "" # "_sqrt" # "_quad" #  

In [4]:
# MMD high dim
dims = [2, 4, 25, 50, 100, 250, 500, 1000, 2000]
ns = [50] * len(dims)

# MMD low dim
dims_ld = [2]
# dims_ld = [2, 4, 25, 50, 100, 250, 500, 1000, 2000]
ns_ld = [1000] * len(dims_ld)

# Load results

In [40]:
# load analytical res
res_analytical_save = pd.read_csv(
    f"{DIR}/res_analytical_{SUFFIX}.csv",
)
res_analytical = res_analytical_save.set_index("dim", drop=False)

In [41]:
# load empirical results
stats_res_list = pickle.load(
    open(f"{DIR}/stats_res_rep_{SUFFIX}.p", "rb"),
)

dims = list(stats_res_list[0].keys())

In [42]:
# load empirical results
# low dim
stats_res_list_ld = pickle.load(
    open(f"{DIR}/stats_res_rep_{SUFFIX}_ld.p", "rb"),
)

dims_ld = list(stats_res_list_ld[0].keys())

# Initialise result classes

In [53]:
# high dim
d = dims[-2]
print("dim:", d)
ts = res_analytical.loc[d, "expectation"] * np.linspace(0., 2., 101)

hd_limits_rep_dims = hd.LimitDistExperimentRepeatedMultiDims(
    empirical_vals_dims_list=stats_res_list,
    res_analytical=res_analytical,
    dims=dims,
    ns=ns,
    ts=ts,
)

dim: 1000


In [72]:
# low dim
d_ld = 2
print("dim:", d_ld)
ts_ld = res_analytical.loc[d_ld, "expectation"] * np.linspace(0.9, 1.1, 101)

hd_limits_rep_dims_ld = hd.LimitDistExperimentRepeatedMultiDims(
    empirical_vals_dims_list=stats_res_list_ld,
    res_analytical=res_analytical,
    dims=dims_ld,
    ns=ns_ld,
    ts=ts_ld,
)

dim: 2


## CDF

In [55]:
cdf_legend = {
    "probs": "Exceeding prob", 
    "cond": "Classic", 
    "full": "High dim Gaussian",
    "mm_full": "High dim Gamma",
}

In [56]:
# compute CDFs
cdf_rep = hd_limits_rep_dims.run(dim=d)

# cdf_rep.replace(cdf_legend, inplace=True)

In [73]:
# low dim
# compute CDFs
cdf_rep_ld = hd_limits_rep_dims_ld.run(dim=d_ld)

# cdf_rep_ld.replace(cdf_legend, inplace=True)

## PDF

In [58]:
pdf_legend = {
    "probs": "Dn", 
    "cond": "Classic", 
    "full": "High dim Gaussian",
    "mm_full": "High dim Gamma",
#     "mm_chi": "High dim Chi-sq",
}

In [59]:
samples_res = hd_limits_rep_dims.get_exp(d).sample_all_limits(size=1000)
# samples_res.replace(pdf_legend, inplace=True)

In [74]:
samples_res_ld = hd_limits_rep_dims_ld.get_exp(d_ld).sample_all_limits(size=1000)
# samples_res_ld.replace(pdf_legend, inplace=True)

## CDF and density plots

1. Check if the eCDF for low-dim is wrong, as it deiates a lot from the normal limit. 

In [61]:
# figs, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))

# linestyles = ["solid", "dashed", "dashdot", "dotted"]
# lw = 3

# # 1.ecdf plots
# cdf_legends = ["Exceeding prob", "Classic", "High dim Gaussian", "High dim Gamma"]
# cdf_legends_show = [r"U-statistic $D_n$", "Classic", "High-D Gaussian", "High-D Gamma"]

# # i. ksd ld
# # sns.lineplot(
# #     ax=axes[0][0],
# #     data=cdf_rep_ld.loc[
# #         cdf_rep_ld.name.isin(cdf_legends)
# #     ], 
# #     x="ts", 
# #     y="value", 
# #     hue="name",
# #     hue_order=cdf_legends,
# #     style="name",
# #     style_order=cdf_legends,
# #     linewidth=2.5,
# # )
# for i, n in enumerate(cdf_legends):
#     sns.lineplot(
#         ax=axes[0][0],
#         data=cdf_rep_ld.loc[cdf_rep_ld.name == n], 
#         x="ts", 
#         y="value", 
#         label=cdf_legends_show[i],
#         linestyle=linestyles[i],
#         linewidth=lw,
#     )

# # statistic expectation
# axes[0][0].axvline(
#     res_analytical.loc[d_ld, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
# )

# axes[0][0].set_xlabel("$t$", fontsize=25)
# axes[0][0].set_ylabel("Probability", fontsize=15)
# axes[0][0].set_ylim(0., 1.01)
# axes[0][0].tick_params(labelsize=15)
# axes[0][0].legend([],[], frameon=False)

# # ii. ksd
# # sns.lineplot(
# #     ax=axes[0][1],
# #     data=cdf_rep.loc[
# #         cdf_rep.name.isin(cdf_legends)
# #     ], 
# #     x="ts", 
# #     y="value", 
# #     hue="name",
# #     style="name",
# #     style_order=cdf_legends,
# #     linewidth=2.5,
# # )
# for i, n in enumerate(cdf_legends):
#     sns.lineplot(
#         ax=axes[0][1],
#         data=cdf_rep.loc[cdf_rep.name == n], 
#         x="ts", 
#         y="value", 
#         label=cdf_legends_show[i],
#         linestyle=linestyles[i],
#         linewidth=lw,
#     )
# # statistic expectation
# axes[0][1].axvline(
#     res_analytical.loc[d, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
# )

# axes[0][1].set_xlabel("$t$", fontsize=25)
# axes[0][1].set_ylim(0., 1.01)
# axes[0][1].tick_params(labelsize=15)
# axes[0][1].ticklabel_format(style="sci", axis="x", scilimits=(0, 0), useMathText=True)
# axes[0][1].xaxis.get_offset_text().set_fontsize(15)
# axes[0][1].get_yaxis().set_visible(False)
# axes[0][1].legend([],[], frameon=False)

# # 2. density plots
# legend_labels = {
#     "Dn": "$D_n$", "Classic": "Classic", "High dim Gaussian": "High dim Gaussian", 
#     "High dim Gamma": "High dim Gamma",
# }

# # i. ksd ld
# for i, (n, l) in enumerate(legend_labels.items()):
#     sns.kdeplot(
#         ax=axes[1][0],
#         data=samples_res_ld.loc[samples_res_ld.name == n],
#         x="x",
#         linewidth=lw,
#         common_norm=False,
#         label=l,
#         linestyle=linestyles[i],
#         legend=False,
#     )
# # statistic expectation
# axes[1][0].axvline(
#     res_analytical.loc[d_ld, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
# )
# axes[1][0].set_xlabel("$x$", fontsize=25)
# axes[1][0].set_ylabel("Density", fontsize=15)
# axes[1][0].set_ylim(bottom=0.)
# axes[1][0].tick_params(labelsize=15)
# axes[1][0].set_xlim([0.35, 0.45])
# # axes[1][0].legend(
# #     fontsize=14, bbox_to_anchor=(2.15, 1.25), ncol=4, #columnspacing=0.7, handletextpad=0.01
# # )

# # ii. ksd
# for i, (n, l) in enumerate(legend_labels.items()):
#     sns.kdeplot(
#         ax=axes[1][1],
#         data=samples_res.loc[samples_res.name == n],
#         x="x",
#         linewidth=lw,
#         linestyle=linestyles[i],
#         common_norm=False,
#         legend=False,
#     )

# # statistic expectation
# axes[1][1].axvline(
#     res_analytical.loc[d, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
# )
# axes[1][1].set_xlabel("$x$", fontsize=25)
# axes[1][1].get_yaxis().set_visible(False)
# axes[1][1].tick_params(labelsize=15)

# axes[0][0].legend(
# #     cdf_legends_show,
#     fontsize=15, bbox_to_anchor=(2.5, 1.42), ncol=len(cdf_legends)+1, #columnspacing=0.7, handletextpad=0.01
# )

# # axes[0][2].legend(
# # #     cdf_legends_show,
# #     fontsize=15, bbox_to_anchor=(1., 1.1), 
# # )

# plt.subplots_adjust(
#     top=0.8,
#     wspace=0.1,
#     hspace=0.6,
# )

# plt.savefig(f"../figs/high_dim/power_{STATISTIC}_{KERNEL.__name__}.pdf", bbox_inches="tight")

In [ ]:
figs, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 2))

linestyles = ["solid", "dashed", "dashdot", "dotted"]
lw = 4

# 1.ecdf plots
cdf_legends_labels = {
    "probs": r"U-statistic $D_n$",
    "cond": "Non-degen.", 
    "mm_full": "Degen. Gamma",
#     "mm_chi": "Degen. Chi-square",
}
    
# i. ksd ld
# sns.lineplot(
#     ax=axes[0][0],
#     data=cdf_rep_ld.loc[
#         cdf_rep_ld.name.isin(cdf_legends)
#     ], 
#     x="ts", 
#     y="value", 
#     hue="name",
#     hue_order=cdf_legends,
#     style="name",
#     style_order=cdf_legends,
#     linewidth=2.5,
# )
for i, (n, l) in enumerate(cdf_legends_labels.items()):
    sns.lineplot(
        ax=axes[0],
        data=cdf_rep_ld.loc[cdf_rep_ld.name == n], 
        x="ts", 
        y="value", 
        label=l,
        linestyle=linestyles[i],
        linewidth=lw,
    )

# statistic expectation
axes[0].axvline(
    res_analytical.loc[d_ld, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
)

axes[0].set_xlabel("$t$", fontsize=25)
axes[0].set_ylabel("Probability", fontsize=20)
axes[0].set_ylim(0., 1.01)
axes[0].tick_params(labelsize=15)
axes[0].legend([],[], frameon=False)

# ii. ksd
# sns.lineplot(
#     ax=axes[0][1],
#     data=cdf_rep.loc[
#         cdf_rep.name.isin(cdf_legends)
#     ], 
#     x="ts", 
#     y="value", 
#     hue="name",
#     style="name",
#     style_order=cdf_legends,
#     linewidth=2.5,
# )
for i, (n, l) in enumerate(cdf_legends_labels.items()):
    sns.lineplot(
        ax=axes[1],
        data=cdf_rep.loc[cdf_rep.name == n], 
        x="ts", 
        y="value", 
        label=l,
        linestyle=linestyles[i],
        linewidth=lw,
    )
# statistic expectation
axes[1].axvline(
    res_analytical.loc[d, "expectation"], color="k", linestyle="--", label=r"Mean $D$", linewidth=2,
)

axes[1].set_xlabel("$t$", fontsize=25)
axes[1].set_ylabel("")
axes[1].set_ylim(0., 1.01)
axes[1].get_yaxis().set_visible(False)
axes[1].tick_params(labelsize=15)
# axes[1].ticklabel_format(style="sci", axis="x", scilimits=(0, 0), useMathText=True)
# axes[1].xaxis.get_offset_text().set_fontsize(15)
axes[1].legend([],[], frameon=False)
pyplot.locator_params(axis="x", nbins=4) # set no. of xticks 

axes[0].legend(
#     cdf_legends_show,
    fontsize=15, bbox_to_anchor=(2.28, 1.25), ncol=4, #columnspacing=0.7, handletextpad=0.01
)
# axes[1].legend(
# #     cdf_legends_show,
#     fontsize=15, bbox_to_anchor=(1.05, 1.), 
# )

plt.subplots_adjust(
    top=0.8,
    wspace=0.1,
    hspace=0.15,
)

plt.savefig(f"../figs/high_dim/power_{STATISTIC}_{KERNEL.__name__}.pdf", bbox_inches="tight")